## Breaking data by state

In [1]:
%%time
%pwd
from pyspark import SparkContext
sc = SparkContext(pyFiles=['/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Section2-PCA/PCA/data_preparation/lib/numpy_pack.py'])

N=sc.defaultParallelism
print('Number of executors=',N)

from pyspark import SparkContext
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

Number of executors= 2
CPU times: user 204 ms, sys: 12 ms, total: 216 ms
Wall time: 13.3 s


In [2]:
#import unpacking routine
import sys
sys.path.append('./lib')
from numpy_pack import unpackArray

import numpy as np

In [3]:
STAT_Descriptions=[
('SortedVals', 'Sample of values', 'vector whose length varies between measurements'),
 ('UnDef', 'sample of number of undefs per row', 'vector whose length varies between measurements'),
 ('mean', 'mean value', ()),
 ('std', 'std', ()),
 ('low100', 'bottom 1%', ()),
 ('high100', 'top 1%', ()),
 ('low1000', 'bottom 0.1%', ()),
 ('high1000', 'top 0.1%', ()),
 ('E', 'Sum of values per day', (365,)),
 ('NE', 'count of values per day', (365,)),
 ('Mean', 'E/NE', (365,)),
 ('O', 'Sum of outer products', (365, 365)),
 ('NO', 'counts for outer products', (365, 365)),
 ('Cov', 'O/NO', (365, 365)),
 ('Var', 'The variance per day = diagonal of Cov', (365,)),
 ('eigval', 'PCA eigen-values', (365,)),
 ('eigvec', 'PCA eigen-vectors', (365, 365))
  ]

In [4]:
US_Weather_parquet='/weather/US_weather.parquet'
measurements=['TMAX','TMIN','TOBS','SNOW','SNWD','PRCP']
Query="SELECT * FROM parquet.`%s`"%US_Weather_parquet
print(Query)

from time import time
t=time()
df = sqlContext.sql(Query).cache()
print(df.count())
print('took',time()-t,'seconds')

SELECT * FROM parquet.`/weather/US_weather.parquet`
3259494
took 22.66908288002014 seconds


In [5]:
US_stations_parquet='/weather/US_stations.parquet'
Query="SELECT * FROM parquet.`%s`"%US_stations_parquet
print(Query)

t=time()
stations = sqlContext.sql(Query).cache()
print(stations.count())
print('took',time()-t,'seconds')

SELECT * FROM parquet.`/weather/US_stations.parquet`
12140
took 0.7413763999938965 seconds


In [6]:
station_state=stations.select(['Station','State'])
station_state.show(4)

+-----------+-----+
|    Station|State|
+-----------+-----+
|USC00341900|   OK|
|USC00428114|   UT|
|USC00165926|   LA|
|USC00411974|   TX|
+-----------+-----+
only showing top 4 rows



In [7]:
df.show(2)

+-----------+-----------+----+--------------------+
|    Station|Measurement|Year|              Values|
+-----------+-----------+----+--------------------+
|CA001126150|       PRCP|1941|[00 7E 00 7E 00 7...|
|CA001126150|       PRCP|1942|[00 00 80 4A 00 0...|
+-----------+-----------+----+--------------------+
only showing top 2 rows



In [8]:
df1=df.join(station_state,on='Station', how='inner')
df1.show(2)

+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|CA001126150|       PRCP|1941|[00 7E 00 7E 00 7...|  NaN|
|CA001126150|       PRCP|1942|[00 00 80 4A 00 0...|  NaN|
+-----------+-----------+----+--------------------+-----+
only showing top 2 rows



In [9]:
records_per_state_pdf=df1.groupby('State').count().toPandas()

records_per_state_pdf = records_per_state_pdf.sort_values('count')
records_per_state_pdf

,State,count
54,NB,120
5,DC,149
29,MB,193
44,AB,594
43,SK,740
8,QC,1218
12,BC,1366
42,ON,1857
9,RI,2608
25,DE,5277


In [10]:
states=list(records_per_state_pdf['State'])
','.join(states)

'NB,DC,MB,AB,SK,QC,BC,ON,RI,DE,CT,VT,NH,NJ,ME,MA,MD,SC,KY,LA,MS,AL,NV,WV,TN,AR,VA,FL,GA,IN,OH,NaN,ND,WY,ID,SD,NC,OK,IL,MN,MO,PA,UT,IA,WI,NY,KS,MI,NE,WA,CO,AZ,NM,OR,MT,CA,TX'

In [12]:
!hdfs dfs -mkdir /weather/by_state

In [24]:
t=time()
for state in states:
    state_df=df1.filter(df1.State==state)
    outfilename="hdfs:///weather/by_state/%s.parquet"%state
    state_df.write.save(outfilename)
    print('done with %s, took %4.2f seconds'%(state,time()-t))
    t=time()

done with NB, took 0.69 seconds
done with DC, took 0.59 seconds
done with MB, took 0.69 seconds
done with AB, took 0.62 seconds
done with SK, took 0.74 seconds
done with QC, took 0.60 seconds
done with BC, took 0.75 seconds
done with ON, took 0.64 seconds
done with RI, took 0.62 seconds
done with DE, took 0.71 seconds
done with CT, took 0.68 seconds
done with VT, took 0.69 seconds
done with NH, took 0.69 seconds
done with NJ, took 0.81 seconds
done with ME, took 0.84 seconds
done with MA, took 0.67 seconds
done with MD, took 0.78 seconds
done with SC, took 0.76 seconds
done with KY, took 0.71 seconds
done with LA, took 0.77 seconds
done with MS, took 0.80 seconds
done with AL, took 0.81 seconds
done with NV, took 0.77 seconds
done with WV, took 0.77 seconds
done with TN, took 0.80 seconds
done with AR, took 0.81 seconds
done with VA, took 0.76 seconds
done with FL, took 0.82 seconds
done with GA, took 0.78 seconds
done with IN, took 0.78 seconds
done with OH, took 0.85 seconds
done wit

In [26]:
!hdfs dfs -du -h /weather/by_state/

273.7 K  /weather/by_state/AB.parquet
13.6 M   /weather/by_state/AL.parquet
16.1 M   /weather/by_state/AR.parquet
23.3 M   /weather/by_state/AZ.parquet
617.9 K  /weather/by_state/BC.parquet
45.9 M   /weather/by_state/CA.parquet
27.2 M   /weather/by_state/CO.parquet
4.1 M    /weather/by_state/CT.parquet
66.8 K   /weather/by_state/DC.parquet
1.7 M    /weather/by_state/DE.parquet
16.2 M   /weather/by_state/FL.parquet
16.0 M   /weather/by_state/GA.parquet
26.6 M   /weather/by_state/IA.parquet
21.9 M   /weather/by_state/ID.parquet
22.4 M   /weather/by_state/IL.parquet
18.4 M   /weather/by_state/IN.parquet
24.0 M   /weather/by_state/KS.parquet
12.8 M   /weather/by_state/KY.parquet
12.9 M   /weather/by_state/LA.parquet
9.3 M    /weather/by_state/MA.parquet
96.4 K   /weather/by_state/MB.parquet
10.9 M   /weather/by_state/MD.parquet
9.3 M    /weather/by_state/ME.parquet
28.1 M   /weather/by_state/MI.parquet
24.6 M   /weather/by_state/MN.parquet
23.3 M   /weather/by_state/MO.parquet
13.6 M   /we

In [27]:
t=time()
%cd /mnt/workspace/Data
!mkdir by_state
%pwd
!ls

/mnt/workspace/Data
by_state  US_stations.parquet  US_weather.parquet


In [72]:
%cd /mnt/workspace/Data/by_state/
t=time()
for state in states:
    name=state+'.parquet'
    !hadoop fs -copyToLocal /weather/by_state/$name $name
    command="tar -czf %s.tgz %s.parquet/"%(state,state)
    print(command)
    !$command
    name=state+'.tgz'
    !aws s3 cp  ./$name s3://mas-dse-open/Weather/by_state/$name
    print('done with %s, took %4.2f seconds'%(state,time()-t))
    t=time()


/mnt/workspace/Data/by_state
copyToLocal: `NB.parquet/NB.parquet/_SUCCESS': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00000-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00001-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00002-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00003-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00004-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00005-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00006-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c000.snappy.parquet': File exists
copyToLocal: `NB.parquet/NB.parquet/part-00007-554426a4-4f31-4ead-8ef3-a4ffb02c35fc-c0

upload: ./CO.tgz to s3://mas-dse-open/Weather/by_state/CO.tgz      
done with CO, took 7.09 seconds
tar -czf AZ.tgz AZ.parquet/
upload: ./AZ.tgz to s3://mas-dse-open/Weather/by_state/AZ.tgz      
done with AZ, took 8.14 seconds
tar -czf NM.tgz NM.parquet/
upload: ./NM.tgz to s3://mas-dse-open/Weather/by_state/NM.tgz      
done with NM, took 7.17 seconds
tar -czf OR.tgz OR.parquet/
upload: ./OR.tgz to s3://mas-dse-open/Weather/by_state/OR.tgz       
done with OR, took 7.31 seconds
tar -czf MT.tgz MT.parquet/
upload: ./MT.tgz to s3://mas-dse-open/Weather/by_state/MT.tgz      
done with MT, took 7.95 seconds
tar -czf CA.tgz CA.parquet/
upload: ./CA.tgz to s3://mas-dse-open/Weather/by_state/CA.tgz      
done with CA, took 8.73 seconds
tar -czf TX.tgz TX.parquet/
upload: ./TX.tgz to s3://mas-dse-open/Weather/by_state/TX.tgz      
done with TX, took 8.88 seconds


In [67]:
!ls -F

NB.parquet/  NB.tgz


In [70]:
command="tar -czf %s.tgz %s.parquet/"%(state,state)
print(command)
!$command
!du -sh *

tar -czf NB.tgz NB.parquet/
120K	NB.parquet
52K	NB.tgz


In [43]:
!du -h *

120K	NB.parquet
52K	NB.tgz


In [58]:
name='NB.tgz'
!aws s3 cp  ./$name s3://mas-dse-open/Weather/by_state/$name

upload: ./NB.tgz to s3://mas-dse-open/Weather/by_state/NB.tgz    


In [73]:
!aws s3 ls mas-dse-open/Weather/by_state/

2018-03-16 20:21:36     220190 AB.tgz
2018-03-16 20:23:05   11432311 AL.tgz
2018-03-16 20:23:31   13377712 AR.tgz
2018-03-16 20:26:36   19254643 AZ.tgz
2018-03-16 20:21:47     506076 BC.tgz
2018-03-16 20:27:08   38154057 CA.tgz
2018-03-16 20:26:29   22868140 CO.tgz
2018-03-16 20:22:04    3408494 CT.tgz
2018-03-16 20:21:29      48306 DC.tgz
2018-03-16 20:21:59    1399298 DE.tgz
2018-03-16 20:23:44   13713886 FL.tgz
2018-03-16 20:23:51   13387038 GA.tgz
2018-03-16 20:25:38   22251334 IA.tgz
2018-03-16 20:24:32   18411253 ID.tgz
2018-03-16 20:24:59   18731601 IL.tgz
2018-03-16 20:23:57   15442891 IN.tgz
2018-03-16 20:26:01   19910478 KS.tgz
2018-03-16 20:22:46   10683818 KY.tgz
2018-03-16 20:22:52   10806071 LA.tgz
2018-03-16 20:22:29    7784319 MA.tgz
2018-03-16 20:21:32      73460 MB.tgz
2018-03-16 20:22:34    9134524 MD.tgz
2018-03-16 20:22:24    7836132 ME.tgz
2018-03-16 20:26:08   23642776 MI.tgz
2018-03-16 20:25:10   20661045 MN.tgz
2018-03-16 20:25:17   19456288 MO.tgz
2018-03-16 2